<a href="https://colab.research.google.com/github/ValterSantos23/Evolucao_Software_2025-2_langextract_atividade2/blob/main/src/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U transformers bitsandbytes accelerate

In [ ]:
import os

!git clone https://github.com/google/langextract.git || true

# Extrair metadados técnicos
branches = !cd langextract && git branch -a
tags = !cd langextract && git tag -l | tail -n 12
history = !cd langextract && git log --merges --oneline -n 15

# Consolidar em uma variável de contexto
resumo_projeto = f"""
Dados extraídos do repositório 'langextract':
1. Branches detectadas: {branches}
2. Últimas Tags (Releases): {tags}
3. Padrão de Merges (Workflow): {history}
"""

print("Dados do projeto extraídos com sucesso!")
print(resumo_projeto)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

model_id = "NousResearch/Hermes-2-Pro-Mistral-7B"
#model_id = "HuggingFaceH4/zephyr-7b-beta"
#model_id = "mistralai/Mistral-7B-Instruct-v0.3"
#model_id = "meta-llama/Llama-3.2-3B-Instruct"
#model_id = "deepseek-ai/deepseek-coder-6.7b-instruct"
#model_id = "Qwen/Qwen2.5-7B-Instruct"

# Configuração para rodar na GPU gratuita do Colab
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
print(f"Modelo {model_id} carregado!")

In [ ]:
from datetime import datetime

# Extrair tags com datas
tags_com_datas = !cd langextract && git for-each-ref \
  --sort=creatordate \
  --format='%(refname:short)|%(creatordate:iso8601)' refs/tags

# Converter para lista estruturada
tags_datas = []
for linha in tags_com_datas:
    tag, data = linha.split("|")
    tags_datas.append((tag.strip(), data.strip()))

tags_datas[:10]

In [ ]:
from datetime import datetime
from statistics import mean, stdev

# Converter datas
datas = [datetime.fromisoformat(d) for _, d in tags_datas]

# Intervalos em horas (mais preciso para releases no mesmo dia)
intervalos_horas = [
    (datas[i] - datas[i-1]).total_seconds() / 3600
    for i in range(1, len(datas))
]

resumo_intervalos = {
    "quantidade_releases": len(datas),
    "intervalo_medio_horas": round(mean(intervalos_horas), 2),
    "intervalo_min_horas": round(min(intervalos_horas), 2),
    "intervalo_max_horas": round(max(intervalos_horas), 2),
    "desvio_padrao_horas": round(stdev(intervalos_horas), 2) if len(intervalos_horas) > 1 else 0
}

resumo_intervalos

In [ ]:
import re
from IPython.display import display, Markdown

def limpar_ansi(texto):
    """Remove códigos de cores e formatação do terminal (ANSI)"""
    return re.sub(r'\x1b\[[0-9;]*m', '', str(texto))

# 1. Limpando os dados do repositório para a IA e para a exibição
branches_limpas = [limpar_ansi(b) for b in branches[:15]] # Limita a 15 branches
tags_limpas = [limpar_ansi(t) for t in tags]
history_limpo = [limpar_ansi(h) for h in history]

# 2. Criando o contexto limpo
contexto_limpo = f"""
Repositório: google/langextract

Tags com datas (ordem cronológica):
{tags_datas}

Estatísticas descritivas dos intervalos entre releases:
- Total de releases: {resumo_intervalos['quantidade_releases']}
- Intervalo médio: {resumo_intervalos['intervalo_medio_horas']} horas
- Intervalo mínimo: {resumo_intervalos['intervalo_min_horas']} horas
- Intervalo máximo: {resumo_intervalos['intervalo_max_horas']} horas
- Desvio padrão do intervalo: {resumo_intervalos['desvio_padrao_horas']} horas

Branches:
{branches_limpas}

Merges:
{history_limpo}
"""


# 3. Prompt estruturado
prompt = f"""
[INST]
Atue como um Consultor Especialista em Governança de Software e Engenharia de Release.

Analise EXCLUSIVAMENTE os dados reais extraídos do repositório GitHub do projeto "langextract", apresentados a seguir:

{contexto_limpo}

 Regras obrigatórias:
- Não assuma práticas que não estejam sustentadas por evidências técnicas observáveis.
- Diferencie evidências diretas (branches, tags, merges, datas) de inferências analíticas.
- Sempre descarte explicitamente os modelos que NÃO se aplicam, com justificativa técnica.
- Utilize métricas temporais (média, mínimo, máximo e variabilidade dos intervalos) quando disponíveis.

---

## MODELOS DE FLUXO DE TRABALHO A CONSIDERAR

Avalie e compare explicitamente os seguintes modelos:

1. GitHub Flow
   - Branch principal única (main)
   - Branches de feature de curta duração
   - Integração frequente via merge para a branch principal
   - Ausência de branches permanentes de release ou develop

2. Gitflow
   - Branches permanentes: main e develop
   - Branches específicas de release e hotfix
   - Ciclo de integração mais longo e estruturado

3. Trunk-Based Development
   - Commits frequentes diretamente na branch principal
   - Branches extremamente curtas ou inexistentes
   - Forte ênfase em integração contínua

---

## ESTRATÉGIAS DE RELEASE A CONSIDERAR

Avalie e compare explicitamente as seguintes estratégias:

1. Continuous Delivery
   - Releases frequentes e potencialmente automatizadas
   - Baixa variabilidade temporal entre releases
   - Relação próxima entre commits e releases

2. Time-Based Release
   - Releases em intervalos temporais relativamente regulares
   - Baixa dispersão estatística dos intervalos

3. Feature-Based Release
   - Releases irregulares
   - Alta variabilidade temporal
   - Releases associadas à conclusão de funcionalidades

4. Rapid Releases
   - Releases muito frequentes (horas ou poucos dias)
   - Baixo intervalo médio e baixo tempo entre versões consecutivas

5. Release Train
   - Releases em ciclos fixos e previsíveis
   - Funcionalidades que não ficam prontas “embarcam no próximo trem”

6. LTS + Current
   - Existência de versões de longo suporte (LTS)
   - Manutenção paralela de versões estáveis e versões correntes
   - Evidência de suporte prolongado em tags ou branches

---

## FORMATO OBRIGATÓRIO DE RESPOSTA

Estruture sua resposta EXATAMENTE da seguinte forma:

# RELATÓRIO DE GOVERNANÇA: LANGEXTRACT

---

## 1. MODELO DE FLUXO DE TRABALHO
Identificação: [Nome do modelo identificado]

Evidências Técnicas:
- [Liste apenas evidências observáveis do repositório]

Análise Comparativa:
- GitHub Flow: [Aderente ou não + justificativa]
- Gitflow: [Aderente ou não + justificativa]
- Trunk-Based Development: [Aderente ou não + justificativa]

Justificativa Final:
[Explique por que o modelo identificado é o mais adequado]

---

## 2. ESTRATÉGIA DE RELEASE
Identificação: [Nome da estratégia identificada]

Evidências Técnicas:
- [Métricas temporais, padrão de tags, cadência observada]

Análise Comparativa:
- Continuous Delivery: [Aderente ou não + justificativa]
- Time-Based Release: [Aderente ou não + justificativa]
- Feature-Based Release: [Aderente ou não + justificativa]
- Rapid Releases: [Aderente ou não + justificativa]
- Release Train: [Aderente ou não + justificativa]
- LTS + Current: [Aderente ou não + justificativa]

Justificativa Final:
[Explique por que a estratégia identificada é a mais compatível]

---

## 3. CONCLUSÃO SOBRE A GOVERNANÇA
[Avaliação objetiva da maturidade da governança do projeto, destacando pontos fortes, limitações e ausência de padronização, quando aplicável]
[/INST]
"""


# 4. Gerando a resposta
resultado = pipe(prompt, max_new_tokens=800, temperature=0.2)
texto_final = resultado[0]['generated_text']

if "[/INST]" in texto_final:
    resposta_ia = texto_final.split("[/INST]")[1]
else:
    resposta_ia = texto_final

display(Markdown(resposta_ia))



---

# RELATÓRIO DE GOVERNANÇA: LANGEXTRACT

---

## 1. MODELO DE FLUXO DE TRABALHO
Identificação: GitHub Flow

Evidências Técnicas:
- Branches de feature de curta duração e frequentes merge para a branch principal (main)
- Absência de branches permanentes de release ou develop

Análise Comparativa:
- GitHub Flow é adequado para projetos que requerem uma maior agilidade e flexibilidade no desenvolvimento e integração de mudanças. A estrutura única de branch (main) e a frequência de merge sugerem que o projeto possui um fluxo de trabalho em conformidade com esse modelo. Porém, é importante notar a ausência de branches de release ou develop, o que pode dificultar a visualização e gerenciamento de mudanças em diferentes estágios de desenvolvimento.
<pad3>: Gitflow e Trunk-Based Development podem ser mais adequados para cenários que necessitam de uma maior estrutura e controle.

Justificativa Final:
GitHub Flow é a estratégia de fluxo de trabalho mais adequada para o projeto Langextract, dado o ciclo de desenvolvimento e integração frequente observado, bem como a curta duração das branches de feature.

---

## 2. ESTRATÉGIA DE RELEASE
Identificação: Continuous Delivery

Evidências Técnicas:
- Releases frequentes e potencialmente automatizadas
- Baixa variabilidade temporal entre releases
- Relação próxima entre commits e releases

Análise Comparativa:
- Continuous Delivery é adequado para projetos que buscam uma entrega contínua e rápida de mudanças, com um foco na automação e integridade dos processos de build, teste e implantação. A análise das estatísticas dos intervalos entre releases e a observação de tags como 'v1.0.0', 'v1.0.1', 'v1.0.2' sugerem uma abordagem de entrega contínua e regular. Porém, é importante notar que a estratégia Time-Based Release também pode ser adequada para cenários que necessitam de um ciclo de lançamento mais previsível.

Justificativa Final:
Continuous Delivery é a estratégia de release mais adequada para o projeto Langextract, dado o ciclo de lançamento regular e a relação próxima entre commits e releases. Essa abordagem permite ao projeto ter uma maior agilidade na entrega de mudanças e melhorias, enquanto mantém um alto nível de integridade e confiabilidade nos lançamentos.

---

## 3. CONCLUSÃO SOBRE A GOVERNANÇA

A governança do projeto Langextract apresenta características de maturidade moderada. O modelo de fluxo de trabalho GitHub Flow e a estratégia de release Continuous Delivery parecem adequados para o cenário atual